In [1]:
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence

In [2]:
# import chord progressions
chord_progressions = pd.DataFrame(pd.read_pickle('chord_progressions.pickle')).reset_index(drop=True).rename({'chord_progression_C':'progression'}, axis=1)

# add end of progression token to each progression
def append_end(progression):
    progression.append('<EOS>')
    return progression

chord_progressions['progression'] = chord_progressions['progression'].apply(lambda x: x + ['<EOS>'])

In [3]:
# tokens
tokens = set()
for progression in chord_progressions.progression:
    tokens |= set(progression)
tokens = sorted(list(tokens))    
    
# examine tokens
i = 0
for token in tokens:
    print(f'{token:8}', end='\t')
    i += 1
    if i%8 == 0:
        print()
        
print('\n')
print('Number of Tokens: ', len(tokens))

<EOS>   	Am      	Am/E    	Am/F    	Am11    	Am6     	Am7     	Am7/C   	
Am7/G   	Am7add11	Am9     	Amadd9  	Amaj7   	Amaj7sus2	Amaj9   	Ammaj7  	
Bm      	Bm11    	Bm6     	Bm7     	Bm7b5   	Bm9     	Bmaj7   	Bmmaj7  	
C       	C/E     	C/G     	C2      	C4      	C5      	C6      	C6add11 	
C7      	C7add11 	C7sus2  	C7sus4  	C9      	Cadd#11 	Cadd11  	Cadd2   	
Cadd4   	Cadd9   	Caug    	Cflat5  	Csus    	Csus2   	Csus4   	Ddim    	
Dm      	Dm11    	Dm13    	Dm6     	Dm6/F   	Dm7     	Dm7/G   	Dm7add11	
Dm7b5   	Dm9     	Dmadd9  	Dmaj    	Dmaj7   	Dmaj9   	E       	E5      	
E6      	E7      	E7b13   	E7b9    	E7sus   	E7sus4  	E9sus4  	Edim    	
Em      	Em6     	Em7     	Em9     	Emadd9  	Emaj7   	Esus2   	Esus4   	
F       	F/G     	F2      	F5      	F6      	F6add9  	F6sus2  	F7      	
F7sus4  	F9      	F9b5    	Fadd#11 	Fadd2   	Fadd4   	Fadd9   	Fflat5  	
Fsus    	Fsus2   	Fsus4   	G       	G#m     	G#m7    	G#maj7  	G/C     	
G11     	G13     	G2      	G4      	G5      	G6   

In [4]:
# tokenizing / encoding

encoder = dict()
idx = 0
for token in tokens:
    encoder[token] = idx
    idx += 1
    
decoder = {v:k for k,v in encoder.items()}

chord_progressions['encoded'] = chord_progressions.applymap(lambda cell: [encoder[chord] for chord in cell])

In [6]:
# max chords in progression -> sequence_length

sequence_length = chord_progressions.progression.str.len().max()
sequence_length

11

In [7]:
# preprocessing

def zero_pad(lst, N=10):
    lst += [0] * (N - len(lst))
    return lst

def create_sliding_window_sequences(chord_progressions, window_size_min, window_size_max, zero_padding=10):
    X, y = [], []
    
    for window_size in range(window_size_min, window_size_max+1):
        for progression in chord_progressions:
            if len(progression) > window_size:
                for i in range(len(progression) - window_size):
                    seq = progression[i:i + window_size]
                    next_chord = progression[i + window_size]

                    X.append(zero_pad(seq, zero_padding))
                    y.append(next_chord)

    return X, y

X, y = create_sliding_window_sequences(chord_progressions['encoded'], 3, sequence_length-1, zero_padding=sequence_length)
len(X), len(y)

(594128, 594128)

In [9]:
# # Transformer Model - PyTorch

# Create a PyTorch dataset
X_tensor = torch.LongTensor(X)  # X is a list of input sequences
y_tensor = torch.LongTensor(y)  # y is a list of target chords
dataset = TensorDataset(X_tensor, y_tensor)  # X_tensor and y_tensor should be your training data tensors

# Create a PyTorch data loader
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define model hyperparameters
d_model = 64  # Size of the model embedding
nhead = 4  # Number of attention heads
num_layers = 3  # Number of transformer layers
dim_feedforward = 128  # Size of the feedforward network
vocab_size = len(tokens)

# Define the model
class ChordTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, dim_feedforward):
        super(ChordTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers, dim_feedforward)
        self.fc = nn.Linear(d_model, vocab_size)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x)
        x = self.fc(x)
        return x
    
# Estimate total memory used
total_memory_for_parameters = 2 * ((vocab_size * d_model * 4) + (d_model * vocab_size * 4))
f'{10 * total_memory_for_parameters * batch_size // (1024*1024) / 10} MB'

'3.9 MB'

In [10]:
# Initialize model, loss, and optimizer
model = ChordTransformer(len(encoder), d_model, nhead, num_layers, dim_feedforward)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# TODO:
#  fix the math error - batch size =wrong= batch size * # X columns
#  get the training going with CUDA
#  figure out how to save, run, save, change lr, run, save, etc

In [ ]:
%%time

# Training loop with batch processing
epochs = 100
for epoch in range(epochs):
    for batch_idx, (X_batch, y_batch) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output.view(-1, len(encoder)), y_batch.view(-1))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
